In [ ]:
token = 'Enter the access token'
cookies = 'Enter the cookie that has been generated'

In [1]:
import requests

url = "https://api.linkedin.com/v2/userinfo"

payload = {}
headers = {
    'Authorization': f'Bearer {token}',
    'Cookie': f'{cookies}'
}

response = requests.request("GET", url, headers=headers, data=payload)

print(response.text)


{"sub":"GahLB6_CfP","email_verified":true,"name":"berojgaar Larka","locale":{"country":"US","language":"en"},"given_name":"berojgaar","family_name":"Larka","email":"h66645507@gmail.com"}
